In [1]:
%matplotlib inline
import numpy as np
import pandas as pd

df = pd.read_csv("onlinefraud.csv")
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score


In [4]:
features = df.drop('isFraud', axis=1)

In [5]:
features = features.drop('isFlaggedFraud', axis=1)

In [6]:
categorical_cols = df.select_dtypes(include=['object']).columns

In [7]:
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    features[col] = le.fit_transform(features[col])
    label_encoders[col] = le

In [8]:
df.shape

(6362620, 11)

In [9]:
features.shape

(6362620, 9)

In [10]:
# Replace 'target' with your target column name
target = df['isFraud']


In [11]:
target.shape

(6362620,)

In [12]:
# Step 3: Split the dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)


In [13]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Define a custom evaluation metric to penalize false negatives
def custom_eval_metric(y_pred, dtrain):
    y_true = dtrain.get_label()
    false_negatives = sum((y_true == 1) & (y_pred < 0.5))
    return 'false_negatives', false_negatives

# Train an XGBoost model with custom evaluation metric
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss'
}

model = xgb.train(params, dtrain, num_boost_round=100, evals=[(dtest, 'test')], feval=custom_eval_metric)



[0]	test-logloss:0.43787	test-false_negatives:964.00000
[1]	test-logloss:0.29740	test-false_negatives:957.00000
[2]	test-logloss:0.20796	test-false_negatives:882.00000
[3]	test-logloss:0.14924	test-false_negatives:861.00000
[4]	test-logloss:0.10799	test-false_negatives:860.00000
[5]	test-logloss:0.07887	test-false_negatives:878.00000
[6]	test-logloss:0.05798	test-false_negatives:832.00000
[7]	test-logloss:0.04303	test-false_negatives:849.00000
[8]	test-logloss:0.03213	test-false_negatives:808.00000
[9]	test-logloss:0.02406	test-false_negatives:823.00000
[10]	test-logloss:0.01821	test-false_negatives:825.00000
[11]	test-logloss:0.01392	test-false_negatives:884.00000
[12]	test-logloss:0.01067	test-false_negatives:792.00000
[13]	test-logloss:0.00831	test-false_negatives:796.00000
[14]	test-logloss:0.00656	test-false_negatives:783.00000
[15]	test-logloss:0.00525	test-false_negatives:767.00000
[16]	test-logloss:0.00429	test-false_negatives:766.00000
[17]	test-logloss:0.00352	test-false_nega

In [14]:
# Predict and evaluate
xg_y_pred = model.predict(dtest)


In [18]:
xg_y_pred
xgb_y_pred=xg_y_pred>0.5


In [19]:
xgb_y_pred=xgb_y_pred.astype(int)

In [20]:
xgb_y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [21]:
accuracy = accuracy_score(y_test, xgb_y_pred)
print(f'Accuracy: {accuracy}')
precision = precision_score(y_test, xgb_y_pred)
print(f'Precision: {precision}')
recall = recall_score(y_test, xgb_y_pred)
print(f'Recall: {recall}')

Accuracy: 0.9997605808089539
Precision: 0.9604283054003724
Recall: 0.8472279260780288
